In [148]:
from selenium import webdriver
import numpy as np
import glob
import os
import datetime
import pandas as pd
from bs4 import BeautifulSoup

### Scrapping website

In [181]:
d = webdriver.Chrome()
d.get("https://bitinfocharts.com/")

In [182]:
table = d.find_element_by_id('main_body').find_element_by_tag_name('tbody').get_attribute('innerHTML')
bs = BeautifulSoup(table, "lxml")
d.quit()

In [183]:
df = pd.DataFrame()
df['Columns'] = [k.find('td').getText() for k in bs.find_all('tr')]
for crypto in ['btc', 'eth', 'ltc']:
    df[crypto.upper()] = [p.getText() for p in bs.find_all(class_='c_%s' % crypto)]

### Cleaning dataframe

In [184]:
stopwords = ['BTC', 'ETH', 'LTC']
def remove_chars(x):
    num = re.sub('[^0-9]','', x)
    if len(str(num)) < 3:
        return x
    else:
        return num
    
def convert_number(x):
    return float(x.replace(',',''))

def remove_crypt(x):
    for k in stopwords:
        if k in x:
            return convert_number(x[:x.index(k)])

df.loc[0] = df.loc[0].apply(lambda x:x[:x.index('(')] if '(' in x else x)
df.loc[1] = df.loc[1].apply(remove_chars)
df.loc[2] = df.loc[2].apply(lambda x:convert_number(x[x.find('$')+1:x.find('USD')]) if 'USD' in x else x)
df.loc[3] = df.loc[3].apply(lambda x:convert_number(x[x.find('$')+1:x.find('USD')]) if 'USD' in x else x)
df.iloc[4, 1:] = df.iloc[4, 1:].apply(convert_number)
for p in range(6,10):
    df.iloc[p, 1:] = df.iloc[p, 1:].apply(remove_crypt)

In [185]:
df.head(15)

,Columns,BTC,ETH,LTC
0,,Bitcoin,Ethereum,Litecoin
1,Total,16669937,95763541,53851932
2,Price,7486.84,328.14,63.65
3,Market Capitalization,1.24805e+11,3.14243e+10,3.42755e+09
4,Transactions last 24h,324631,498898,26347
5,Transactions avg. per hour,"13,526","20,787","1,098"
6,Sent last 24h,4.14134e+06,1.45527e+07,2.38957e+07
7,Sent avg. per hour,172556,606362,995655
8,Avg. Transaction Value,12.76,29.17,906.96
9,Median Transaction Value,0.095,0.052,6.03
